# GAIA UI Element Object Detection Training

이 노트북은 GAIA 테스트 시스템에서 수집한 UI 요소 데이터로 YOLO 객체 인식 모델을 학습합니다.

**실행 환경:**
- Google Colab (무료 T4 GPU)
- 또는 로컬 맥북 (Apple Silicon MPS)

**예상 시간:**
- Colab T4: 1-2시간
- M1/M2 맥북: 1-2시간
- M3 맥북: 30분-1시간

## Step 1: 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install ultralytics opencv-python pillow

import os
import torch
from ultralytics import YOLO
from pathlib import Path

# 디바이스 확인
if torch.backends.mps.is_available():
    device = "mps"  # Apple Silicon
    print("✅ Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = "0"  # NVIDIA GPU
    print(f"✅ Using NVIDIA GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("⚠️ Using CPU (will be slow)")

print(f"PyTorch version: {torch.__version__}")

## Step 2: 데이터셋 업로드 (Colab만)

**로컬 맥북에서 실행 시 이 셀은 스킵하세요!**

Colab에서 실행 시:
1. `artifacts/training_data` 폴더를 ZIP으로 압축
2. Google Drive에 업로드
3. 아래 코드로 마운트

In [ ]:
# Google Colab에서만 실행
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # ZIP 압축 해제 (Google Drive에 training_data.zip 업로드한 경우)
    !unzip -q /content/drive/MyDrive/training_data.zip -d /content/
    
    DATA_DIR = "/content/training_data"
    print(f"✅ Data loaded from Google Drive")
except:
    # 로컬 실행
    DATA_DIR = "artifacts/training_data"
    print(f"✅ Using local data: {DATA_DIR}")

# 데이터셋 확인
images_dir = Path(DATA_DIR) / "images"
labels_dir = Path(DATA_DIR) / "labels"

num_images = len(list(images_dir.glob("*.png")))
num_labels = len(list(labels_dir.glob("*.txt")))

print(f"📊 Dataset statistics:")
print(f"   Images: {num_images}")
print(f"   Labels: {num_labels}")

if num_images < 50:
    print("⚠️ WARNING: Less than 50 images! Collect more data for better results.")
elif num_images < 200:
    print("⚠️ 50-200 images: Model will work but accuracy may be limited.")
else:
    print("✅ Good dataset size! Training should produce good results.")

## Step 3: 데이터셋 분할 (Train/Val)

80% 학습, 20% 검증으로 분할

In [ ]:
import random
import shutil

# 학습/검증 폴더 생성
train_images = Path(DATA_DIR) / "images" / "train"
val_images = Path(DATA_DIR) / "images" / "val"
train_labels = Path(DATA_DIR) / "labels" / "train"
val_labels = Path(DATA_DIR) / "labels" / "val"

for folder in [train_images, val_images, train_labels, val_labels]:
    folder.mkdir(parents=True, exist_ok=True)

# 모든 이미지 파일
all_images = list(images_dir.glob("*.png"))
random.shuffle(all_images)

# 80/20 분할
split_idx = int(len(all_images) * 0.8)
train_imgs = all_images[:split_idx]
val_imgs = all_images[split_idx:]

# 파일 복사
for img_path in train_imgs:
    label_path = labels_dir / f"{img_path.stem}.txt"
    shutil.copy(img_path, train_images / img_path.name)
    if label_path.exists():
        shutil.copy(label_path, train_labels / label_path.name)

for img_path in val_imgs:
    label_path = labels_dir / f"{img_path.stem}.txt"
    shutil.copy(img_path, val_images / img_path.name)
    if label_path.exists():
        shutil.copy(label_path, val_labels / label_path.name)

print(f"✅ Dataset split:")
print(f"   Train: {len(train_imgs)} images")
print(f"   Val: {len(val_imgs)} images")

## Step 4: data.yaml 업데이트

In [ ]:
# YOLO data.yaml 생성
yaml_content = f"""# GAIA UI Element Detection Dataset
path: {Path(DATA_DIR).absolute()}
train: images/train
val: images/val

# Classes
names:
  0: button
  1: input
  2: link
  3: checkbox
  4: radio
  5: dropdown
  6: text
  7: image

# Number of classes
nc: 8
"""

yaml_path = Path(DATA_DIR) / "data.yaml"
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Created {yaml_path}")

## Step 5: 모델 학습

**YOLOv8n (nano)**: 가장 빠르고 가벼움, 실시간 추론에 적합  
**YOLOv8s (small)**: 중간 크기, 더 정확함  
**YOLOv8m (medium)**: 큰 크기, 가장 정확하지만 느림

맥북은 nano 추천, Colab은 small/medium 추천

In [ ]:
# 모델 크기 선택
MODEL_SIZE = "yolov8n.pt"  # nano (가장 빠름)
# MODEL_SIZE = "yolov8s.pt"  # small (중간)
# MODEL_SIZE = "yolov8m.pt"  # medium (가장 정확)

# 학습 파라미터
EPOCHS = 50  # 맥북: 30-50, Colab: 50-100
BATCH_SIZE = 16  # 맥북: 8-16, Colab: 16-32
IMG_SIZE = 640  # 이미지 크기

# 모델 로드
model = YOLO(MODEL_SIZE)

# 학습 시작
print(f"🚀 Training started with {MODEL_SIZE}...")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Device: {device}")
print(f"   Expected time: {'1-2 hours' if device == 'mps' else '1-1.5 hours' if device == '0' else '5-10 hours'}")

results = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=device,
    
    # 최적화 설정
    workers=4,  # 데이터 로딩 워커
    cache=True,  # 메모리 캐싱 (빠름)
    amp=True,    # Mixed precision (빠름)
    
    # 저장 설정
    project='runs/detect',
    name='ui_detector',
    exist_ok=True,
    
    # 조기 종료 (과적합 방지)
    patience=10,
    
    # Verbose
    verbose=True,
    
    # 학습 중 시각화
    plots=True
)

print("✅ Training completed!")
print(f"   Best model: {results.save_dir}/weights/best.pt")

## Step 6: 모델 평가

In [ ]:
# 검증 데이터로 평가
metrics = model.val()

print(f"📊 Validation Results:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

if metrics.box.map50 > 0.8:
    print("✅ Excellent! Model is very accurate.")
elif metrics.box.map50 > 0.6:
    print("✅ Good! Model should work well.")
elif metrics.box.map50 > 0.4:
    print("⚠️ Okay. Consider collecting more data or training longer.")
else:
    print("❌ Poor accuracy. Need more/better training data.")

## Step 7: 테스트 추론

In [ ]:
# 테스트 이미지로 추론
test_img = list(val_images.glob("*.png"))[0]

results = model.predict(
    source=str(test_img),
    conf=0.5,  # Confidence threshold
    save=True,  # 결과 이미지 저장
    show_labels=True,
    show_conf=True
)

print(f"✅ Prediction saved to: {results[0].save_dir}")

# 결과 출력
for box in results[0].boxes:
    class_id = int(box.cls[0])
    class_name = model.names[class_id]
    confidence = float(box.conf[0])
    x1, y1, x2, y2 = box.xyxy[0]
    print(f"   {class_name}: {confidence:.2f} at ({x1:.0f}, {y1:.0f})")

## Step 8: 모델 저장 및 다운로드

In [ ]:
# 최종 모델 저장
best_model_path = Path(results.save_dir) / "weights" / "best.pt"
output_path = "ui_detector.pt"

shutil.copy(best_model_path, output_path)
print(f"✅ Model saved to: {output_path}")

# Colab에서 다운로드
try:
    from google.colab import files
    files.download(output_path)
    print("📥 Downloading model...")
except:
    print(f"💾 Model ready at: {Path(output_path).absolute()}")
    print(f"   Copy to: gaia/models/ui_detector.pt")

## Step 9: 속도 벤치마크

In [ ]:
import time
import numpy as np

# 속도 테스트
test_img = str(list(val_images.glob("*.png"))[0])

# Warm up
for _ in range(3):
    model.predict(test_img, verbose=False)

# Benchmark
times = []
for _ in range(20):
    start = time.time()
    model.predict(test_img, verbose=False)
    times.append((time.time() - start) * 1000)  # ms

print(f"⚡ Inference Speed:")
print(f"   Average: {np.mean(times):.1f}ms")
print(f"   Min: {np.min(times):.1f}ms")
print(f"   Max: {np.max(times):.1f}ms")

if np.mean(times) < 50:
    print("✅ Excellent! Real-time capable.")
elif np.mean(times) < 100:
    print("✅ Good! Fast enough for testing.")
else:
    print("⚠️ Slow. Consider using smaller model (yolov8n).")

## 완료! 🎉

**다음 단계:**
1. `ui_detector.pt` 파일을 `gaia/models/`에 복사
2. `ui_detector.py` 통합 코드 실행
3. GAIA 테스트 실행하여 객체 인식 테스트

**예상 성능:**
- 셀렉터 탐색 성공률: 60% → 85%+
- LLM 호출 횟수: 100% → 30%
- 실행 속도: 3배 빠름
- 비용: GPT-4V $10/1000회 → $0